# 说明
- 本代码是在 https://github.com/NM001007/An-Attention-based-Hybrid-Suicide-Ideation-Detection/blob/main/An_attention_based_hybrid_architecture.ipynb 的基础上修改而来
- 是论文 "An attention-based hybrid architecture with explainability for depressive social media text detection in Bangla" 的不完整实现
- 知乎解释文章：https://zhuanlan.zhihu.com/p/25995878591
- Author： Bryce Wang


*论文互助小组*
AI缝合论文发表互助小组(非盈利，纯学术)，欢迎大部分时间都在自我摸索的学术孤儿加入：brycewang2018@gmail.com/brycew6m@stanford.edu。

In [4]:
# prompt: install packages via /content/drive/MyDrive/0-GitHub/An-Attention-based-Hybrid-Suicide-Ideation-Detection/requirements.txt

!pip install -r /content/drive/MyDrive/0-GitHub/An-Attention-based-Hybrid-Suicide-Ideation-Detection/requirements.txt


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 108.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.5/825.5 kB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 108.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.0/278.0 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [5]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
import tensorflow as tf
# from tensorflow.keras.layers import *
# from tensorflow.keras.models import *
from nltk.tokenize import word_tokenize
from keras_preprocessing.sequence import pad_sequences
#from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import keras.backend as K
import gensim
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score

import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords

import warnings
warnings.filterwarnings(action = 'ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
#Preprocessing function
def preprocessing(data_frame):
    ## Preprocessing
    # Removing URLs whithin the tweets
    data_frame["Text"] = data_frame["Text"].str.replace(r'\s*https?://\S+(\s+|$)', ' ').str.strip()
    # Removing emails, hashtags and punctuations
    data_frame['Text'] = data_frame["Text"].str.replace(r'\S*@\S*\s?', ' ').str.strip()
    data_frame['Text'] = data_frame['Text'].str.replace(r'#\S*\s?', ' ').str.strip()
    data_frame['Text'] = data_frame['Text'].str.replace(r'[^\w\s]+', ' ').str.strip()

    # # Removing stopwords
    stop = stopwords.words('english')
    data_frame['Text'].apply(lambda x: [item for item in str(x) if item not in stop])

    # Removing newline characters
    data_frame['Text'] = data_frame['Text'].str.rstrip()

    # Tokenizing Posts and counting the length of each post
    data_frame['Tokens'] = data_frame.apply(lambda row: word_tokenize(str(row['Text'])), axis=1)
    data_frame['Length'] = data_frame.apply(lambda row: len(row['Tokens']), axis=1)

    return data_frame

# Data

In [7]:
#DS_Path = "/Datasets"
#DS_Path = "/Users/brycewang/Documents/GitHub/An-Attention-based-Hybrid-Suicide-Ideation-Detection/Datasets"
DS_Path = "/content/drive/MyDrive/0-GitHub/An-Attention-based-Hybrid-Suicide-Ideation-Detection/Datasets"

In [8]:
#!export NLTK_DATA="/Users/brycewang/nltk_data"

import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

### Twitter 1

In [9]:
# ## Preparing the data
# ## Twitter 10000
Twitter_path = DS_Path + "/Twitter/twitter-suicidal_data_10000.csv"
df = pd.read_csv(Twitter_path, encoding='latin-1')
df = df.rename(columns={'tweet': 'Text', 'intention': 'Label'})
#print(df.columns)
df = preprocessing(df)
df

,Text,Label,Tokens,Length
0,my life is meaningless i just want to end my l...,1,"[my, life, is, meaningless, i, just, want, to,...",79
1,muttering i wanna die to myself daily for a fe...,1,"[muttering, i, wan, na, die, to, myself, daily...",46
2,work slave i really feel like my only purpose ...,1,"[work, slave, i, really, feel, like, my, only,...",69
3,i did something on the 2 of october i overdose...,1,"[i, did, something, on, the, 2, of, october, i...",77
4,i feel like no one cares i just want to die ma...,1,"[i, feel, like, no, one, cares, i, just, want,...",18
...,...,...,...,...
9114,have you ever laid on your bed at night and cr...,1,"[have, you, ever, laid, on, your, bed, at, nig...",33
9115,the fault the blame the pain s still there i m...,1,"[the, fault, the, blame, the, pain, s, still, ...",20
9116,stop asking me to trust you when i m still cou...,1,"[stop, asking, me, to, trust, you, when, i, m,...",22
9117,i never know how to handle sadness crying make...,1,"[i, never, know, how, to, handle, sadness, cry...",12


In [10]:
import nltk
print(nltk.data.path)

['/root/nltk_data', '/usr/nltk_data', '/usr/share/nltk_data', '/usr/lib/nltk_data', '/usr/share/nltk_data', '/usr/local/share/nltk_data', '/usr/lib/nltk_data', '/usr/local/lib/nltk_data']


### Twitter 2

In [11]:
twitter_path = DS_Path + "/Twitter/suicidal-tendency-tweets.csv"
df = pd.read_csv(twitter_path, encoding='latin-1', usecols=['tweet', 'intention'], nrows = 17142)
df = df.rename(columns={'tweet': 'Text', 'intention': 'Label'})

df = preprocessing(df)
df

,Text,Label,Tokens,Length
0,@ModernTosspot @kurtistrippled @Jebadoo2 to go...,1,"[@, ModernTosspot, @, kurtistrippled, @, Jebad...",48
1,@ModernTosspot @kurtistrippled @Jebadoo2 Thy ...,1,"[@, ModernTosspot, @, kurtistrippled, @, Jebad...",45
2,@notsmokinfeds @DirtyDianaDyke @LiberatedLezzy...,0,"[@, notsmokinfeds, @, DirtyDianaDyke, @, Liber...",14
3,@notsmokinfeds @DirtyDianaDyke @LiberatedLezzy...,0,"[@, notsmokinfeds, @, DirtyDianaDyke, @, Liber...",44
4,@Nikki40_40 @AJLIESHERE @littlezupa @kurtistri...,0,"[@, Nikki40_40, @, AJLIESHERE, @, littlezupa, ...",43
...,...,...,...,...
17137,Make sure you are happy in real life. Not just...,0,"[Make, sure, you, are, happy, in, real, life, ...",40
17138,With one of our keyrings you will never be alo...,0,"[With, one, of, our, keyrings, you, will, neve...",23
17139,@jimmac63793836 @HelenMurphy99 Always here if ...,0,"[@, jimmac63793836, @, HelenMurphy99, Always, ...",13
17140,Itâs not selfish. Suicide is usually caused ...,1,"[Itâs, not, selfish, ., Suicide, is, usually...",49


In [12]:
print(list(df['Label']).count(1), list(df['Label']).count(0)) #3754 13388

3754 13388


## Reddit SNS

In [13]:
# ## Preparing the data

Reddit_path = DS_Path + "/Reddit/Suicide_Detection_1.csv"

## reading the abovementioned datasets
# df_r = pd.read_csv(Reddit_path, encoding='latin-1', usecols=['text', 'class'])
df_r = pd.read_csv(Reddit_path, encoding='latin-1', usecols=['text', 'class'], nrows = 20000)
df_r = df_r.rename(columns={'text': 'Text', 'class': 'Label'})

df = preprocessing(df_r)

In [14]:
df.head()

,Text,Label,Tokens,Length
0,Ex Wife Threatening SuicideRecently I left my ...,suicide,"[Ex, Wife, Threatening, SuicideRecently, I, le...",156
1,Am I weird I don't get affected by compliments...,non-suicide,"[Am, I, weird, I, do, n't, get, affected, by, ...",29
2,Finally 2020 is almost over... So I can never ...,non-suicide,"[Finally, 2020, is, almost, over, ..., So, I, ...",31
3,i need helpjust help me im crying so hard,suicide,"[i, need, helpjust, help, me, im, crying, so, ...",9
4,"Iâm so lostHello, my name is Adam (16) and I...",suicide,"[Iâm, so, lostHello, ,, my, name, is, Adam, ...",478


In [15]:
print(df['Label'].isna().sum())  # 查看 NaN 的数量
df = df.dropna(subset=['Label'])  # 删除 NaN 行

df = df[df['Label'].isin(['suicide', 'non-suicide'])]

5


In [16]:
df.head()

,Text,Label,Tokens,Length
0,Ex Wife Threatening SuicideRecently I left my ...,suicide,"[Ex, Wife, Threatening, SuicideRecently, I, le...",156
1,Am I weird I don't get affected by compliments...,non-suicide,"[Am, I, weird, I, do, n't, get, affected, by, ...",29
2,Finally 2020 is almost over... So I can never ...,non-suicide,"[Finally, 2020, is, almost, over, ..., So, I, ...",31
3,i need helpjust help me im crying so hard,suicide,"[i, need, helpjust, help, me, im, crying, so, ...",9
4,"Iâm so lostHello, my name is Adam (16) and I...",suicide,"[Iâm, so, lostHello, ,, my, name, is, Adam, ...",478


In [17]:
label_dict = {'suicide': 1, 'non-suicide': 0}
df['Label'] = df['Label'].apply(lambda row: label_dict[row])

In [18]:
df.loc[19994]

,19994
Text,ahhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh...
Label,0
Tokens,[ahhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh...
Length,5


In [19]:
print(list(df['Label']).count(1), list(df['Label']).count(0)) # print(list(df['Label']).count(1), list(df['Label']).count(0))

9860 10131


## Reddit _ SD

In [20]:
## Preparing the data
reddit_path = DS_Path + "/Reddit/reddit_depression_suicidewatch.csv"

df = pd.read_csv(reddit_path, encoding='latin-1')
df = df.rename(columns={'text':'Text', 'label':'Label'})
label_dict = {'depression': 0, 'SuicideWatch': 1}
df['Label'] = df['Label'].apply(lambda row: label_dict[row])
df = preprocessing(df)
df

,Text,Label,Tokens,Length
0,I recently went through a breakup and she said...,0,"[I, recently, went, through, a, breakup, and, ...",126
1,"I do not know how to navigate these feelings, ...",0,"[I, do, not, know, how, to, navigate, these, f...",327
2,"So I have been with my bf for 5 months , and h...",0,"[So, I, have, been, with, my, bf, for, 5, mont...",144
3,I am so exhausted of this. Just when I think I...,1,"[I, am, so, exhausted, of, this, ., Just, when...",378
4,I have been severly bullied since i was 5 till...,0,"[I, have, been, severly, bullied, since, i, wa...",215
...,...,...,...,...
20358,I took 50mg of seroquel a few hours after I dr...,1,"[I, took, 50mg, of, seroquel, a, few, hours, a...",36
20359,that is what has happened to me last week. And...,0,"[that, is, what, has, happened, to, me, last, ...",130
20360,Ever just feel alone in a house full of people...,0,"[Ever, just, feel, alone, in, a, house, full, ...",22
20361,Politicians. Neighbors. Corporations. Society....,0,"[Politicians, ., Neighbors, ., Corporations, ....",40


## Reddit_SD (Positive Suicide Samples) + Facebook (Negative samples)

In [21]:
## Preparing the data
reddit_path = DS_Path + "/Reddit/reddit_depression_suicidewatch.csv"

df = pd.read_csv(reddit_path, encoding='latin-1')
df = df.rename(columns={'text':'Text', 'label':'Label'})
label_dict = {'depression': 0, 'SuicideWatch': 1}
df['Label'] = df['Label'].apply(lambda row: label_dict[row])

df.drop(df[df['Label'] <1].index, inplace = True)
df.reset_index(drop=True)

## Adding negative samples

zero_sampels = DS_Path + "/Facebook/100_000_samples.csv"

df_zero = pd.read_csv(zero_sampels, encoding='latin-1', nrows=df.shape[0])
df_zero = df_zero.rename(columns={'message':'Text'})
df_zero = df_zero.assign(Label=0)

In [22]:
df

,Text,Label
3,I am so exhausted of this. Just when I think I...,1
5,I am 20 year old with some good friends but I ...,1
8,it is looming around the corner again. It alwa...,1
9,there is.....foodAnd other things I will be ju...,1
13,I am on zoloft and focalin and it is changed m...,1
...,...,...
20353,I am sorry I am sorry i made studying as Joke ...,1
20354,Being told to end my life by the only person I...,1
20356,Last week I did the most disgusting thing by c...,1
20357,I do not see a way out of my head and a way to...,1


In [23]:
df_zero

,Text,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Label
0,We are speaking to NRA supporters as well as W...,NaN,NaN,NaN,NaN,0
1,If you are just joining us we are outside of t...,NaN,NaN,NaN,NaN,0
2,Do you know how backward America are in allowi...,NaN,NaN,NaN,NaN,0
3,People who legally own guns often seem all too...,NaN,NaN,NaN,NaN,0
4,Have you snowflakes watched the news and seen ...,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...
9987,"Gail, there is no mystery, this is just the af...",NaN,NaN,NaN,NaN,0
9988,Had a great time there last year with the chil...,NaN,NaN,NaN,NaN,0
9989,We went last year and it was great fun!!! The ...,NaN,NaN,NaN,NaN,0
9990,Sergiu Petrache Penelope Diaz Manso Karina Tar...,NaN,NaN,NaN,NaN,0


In [24]:
df = pd.concat([df, df_zero])

In [25]:
df.reset_index(drop=True)

,Text,Label,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,I am so exhausted of this. Just when I think I...,1,NaN,NaN,NaN,NaN
1,I am 20 year old with some good friends but I ...,1,NaN,NaN,NaN,NaN
2,it is looming around the corner again. It alwa...,1,NaN,NaN,NaN,NaN
3,there is.....foodAnd other things I will be ju...,1,NaN,NaN,NaN,NaN
4,I am on zoloft and focalin and it is changed m...,1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
19979,"Gail, there is no mystery, this is just the af...",0,NaN,NaN,NaN,NaN
19980,Had a great time there last year with the chil...,0,NaN,NaN,NaN,NaN
19981,We went last year and it was great fun!!! The ...,0,NaN,NaN,NaN,NaN
19982,Sergiu Petrache Penelope Diaz Manso Karina Tar...,0,NaN,NaN,NaN,NaN


In [26]:
df = preprocessing(df)
df

,Text,Label,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Tokens,Length
3,I am so exhausted of this. Just when I think I...,1,NaN,NaN,NaN,NaN,"[I, am, so, exhausted, of, this, ., Just, when...",378
5,I am 20 year old with some good friends but I ...,1,NaN,NaN,NaN,NaN,"[I, am, 20, year, old, with, some, good, frien...",148
8,it is looming around the corner again. It alwa...,1,NaN,NaN,NaN,NaN,"[it, is, looming, around, the, corner, again, ...",117
9,there is.....foodAnd other things I will be ju...,1,NaN,NaN,NaN,NaN,"[there, is, ....., foodAnd, other, things, I, ...",48
13,I am on zoloft and focalin and it is changed m...,1,NaN,NaN,NaN,NaN,"[I, am, on, zoloft, and, focalin, and, it, is,...",179
...,...,...,...,...,...,...,...,...
9987,"Gail, there is no mystery, this is just the af...",0,NaN,NaN,NaN,NaN,"[Gail, ,, there, is, no, mystery, ,, this, is,...",22
9988,Had a great time there last year with the chil...,0,NaN,NaN,NaN,NaN,"[Had, a, great, time, there, last, year, with,...",28
9989,We went last year and it was great fun!!! The ...,0,NaN,NaN,NaN,NaN,"[We, went, last, year, and, it, was, great, fu...",49
9990,Sergiu Petrache Penelope Diaz Manso Karina Tar...,0,NaN,NaN,NaN,NaN,"[Sergiu, Petrache, Penelope, Diaz, Manso, Kari...",28


# Word Embeddings

## Downloading Pre-trained Models

In [27]:
import gensim.downloader as api

import json
info = api.info()
print(info.keys())

for model_name, model_data in sorted(info['models'].items()):
    print(
        '%s (%d records): %s' % (
            model_name,
            model_data.get('num_records', -1),
            model_data['description'][:40] + '...',
        )
    )

dict_keys(['corpora', 'models'])
__testing_word2vec-matrix-synopsis (-1 records): [THIS IS ONLY FOR TESTING] Word vecrors ...
conceptnet-numberbatch-17-06-300 (1917247 records): ConceptNet Numberbatch consists of state...
fasttext-wiki-news-subwords-300 (999999 records): 1 million word vectors trained on Wikipe...
glove-twitter-100 (1193514 records): Pre-trained vectors based on  2B tweets,...
glove-twitter-200 (1193514 records): Pre-trained vectors based on 2B tweets, ...
glove-twitter-25 (1193514 records): Pre-trained vectors based on 2B tweets, ...
glove-twitter-50 (1193514 records): Pre-trained vectors based on 2B tweets, ...
glove-wiki-gigaword-100 (400000 records): Pre-trained vectors based on Wikipedia 2...
glove-wiki-gigaword-200 (400000 records): Pre-trained vectors based on Wikipedia 2...
glove-wiki-gigaword-300 (400000 records): Pre-trained vectors based on Wikipedia 2...
glove-wiki-gigaword-50 (400000 records): Pre-trained vectors based on Wikipedia 2...
word2vec-google-new

In [28]:
embeddings = {0: "word2vec-google-news-300", 1: "fasttext-wiki-news-subwords-300", 2: "glove-twitter-100"}
embedding_format = 1

embedding_model = api.load(embeddings[embedding_format])

[==================================================] 100.0% 958.5/958.4MB downloaded


In [29]:
similar_words = embedding_model.most_similar('kill')
print(similar_words) # [('kills', 0.7860026955604553), ('killing', 0.7399880290031433), ('destroy', 0.7350039482116699), ('maim', 0.7278950810432434), ('decapitate', 0.7270759344100952), ('slay', 0.7220808267593384), ('killy', 0.721636176109314), ('kill-', 0.7112785577774048), ('kill.', 0.705057680606842), ('killed', 0.7015096545219421)]

[('kills', 0.7860026955604553), ('killing', 0.7399880290031433), ('destroy', 0.7350039482116699), ('maim', 0.7278950810432434), ('decapitate', 0.7270759344100952), ('slay', 0.7220808267593384), ('killy', 0.721636176109314), ('kill-', 0.7112785577774048), ('kill.', 0.705057680606842), ('killed', 0.7015096545219421)]


In [30]:
vocab = embedding_model.key_to_index

In [31]:
## Creating the embedding matrix

# vocab = wd2vc_model.wv.vocab ## gensim 3
vocab = embedding_model.key_to_index
print("The total number of words are : ", len(vocab))
vocab = list(vocab.keys())

word_vec_dict = {}
for word in vocab:
    word_vec_dict[word] = embedding_model.get_vector(word)
print("The no of key-value pairs : ", len(word_vec_dict))  # should come equal to vocab size

max_length = max(df['Length'])
print("maximum length = ", max_length)

tok = Tokenizer()
tok.fit_on_texts(df['Tokens'])
vocab_size = len(tok.word_index) + 1

encd_rev = tok.texts_to_sequences(df['Tokens'])

embed_dim = 300
pad_rev = pad_sequences(encd_rev, maxlen=max_length, padding='post')
print("pad_shape = ", pad_rev.shape)

# the embedding matrix
embed_matrix = np.zeros(shape=(vocab_size, embed_dim))
print(embed_matrix.shape)
for word, i in tok.word_index.items():
    embed_vector = word_vec_dict.get(word)
    if embed_vector is not None:
        embed_matrix[i] = embed_vector

The total number of words are :  999999
The no of key-value pairs :  999999
maximum length =  5612
pad_shape =  (19984, 5612)
(43895, 300)


## Train_Validation_Test data

In [32]:
#embed_matrix[1] # (300,) ，一维，没有列的信息
pd.DataFrame(embed_matrix[1]) # .reshape(1, -1)

,0
0,-0.060418
1,0.069955
2,0.053173
3,0.052922
4,0.053978
...,...
295,0.000701
296,-0.127730
297,-0.004779
298,0.033832


In [33]:
## Spliting the data
X, X_test, y, y_test = train_test_split(pad_rev, df['Label'], shuffle=True, test_size=0.2, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=33)

print("Train = ", X_train.shape, y_train.shape)
print("Test = ", X_test.shape, y_test.shape)
print("Validation = ",X_val.shape, y_val.shape)

#Train =  (13032, 9360) (13032,)
#Test =  (4073, 9360) (4073,)
#Validation =  (3258, 9360) (3258,)

Train =  (12789, 5612) (12789,)
Test =  (3997, 5612) (3997,)
Validation =  (3198, 5612) (3198,)


In [34]:
X_train
df_ytrain = pd.DataFrame(y_train)
df_ytrain.Label.value_counts()  # 1    6394

df_ytest = pd.DataFrame(y_test)
df_ytest.Label.value_counts() # 1    1999

,count
Label,
0,2002
1,1995


# Attention_based hybrid BiLSTM - CNN model

In [35]:
class LuongAttention(tf.keras.Model):
    def __init__(self, rnn_size, attention_func):
        super(LuongAttention, self).__init__()
        self.attention_func = attention_func

        if attention_func not in ['dot', 'general', 'concat']:
            raise ValueError(
                'Unknown attention score function! Must be either dot, general or concat.')

        if attention_func == 'general':
            # General score function
            self.wa = tf.keras.layers.Dense(rnn_size)
        elif attention_func == 'concat':
            # Concat score function
            self.wa = tf.keras.layers.Dense(rnn_size, activation='tanh')
            self.va = tf.keras.layers.Dense(1)

    def call(self, decoder_output, encoder_output):
        if self.attention_func == 'dot':
            # Dot score function: decoder_output (dot) encoder_output
            # decoder_output has shape: (batch_size, 1, rnn_size)
            # encoder_output has shape: (batch_size, max_len, rnn_size)
            # => score has shape: (batch_size, 1, max_len)
            score = tf.matmul(decoder_output, encoder_output, transpose_b=True)
        elif self.attention_func == 'general':
            # General score function: decoder_output (dot) (Wa (dot) encoder_output)
            # decoder_output has shape: (batch_size, 1, rnn_size)
            # encoder_output has shape: (batch_size, max_len, rnn_size)
            # => score has shape: (batch_size, 1, max_len)
            score = tf.matmul(decoder_output, self.wa(
                encoder_output), transpose_b=True)
        elif self.attention_func == 'concat':
            # Concat score function: va (dot) tanh(Wa (dot) concat(decoder_output + encoder_output))
            # Decoder output must be broadcasted to encoder output's shape first
            decoder_output = tf.tile(
                decoder_output, [1, encoder_output.shape[1], 1])

            # Concat => Wa => va
            # (batch_size, max_len, 2 * rnn_size) => (batch_size, max_len, rnn_size) => (batch_size, max_len, 1)
            score = self.va(
                self.wa(tf.concat((decoder_output, encoder_output), axis=-1)))

            # Transpose score vector to have the same shape as other two above
            # (batch_size, max_len, 1) => (batch_size, 1, max_len)
            score = tf.transpose(score, [0, 2, 1])

        # alignment a_t = softmax(score)
        alignment = tf.nn.softmax(score, axis=2)

        # context vector c_t is the weighted average sum of encoder output
        context = tf.matmul(alignment, encoder_output)

        return context, alignment

In [36]:
# from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Input, Embedding, Conv1D, GlobalMaxPooling1D, Dropout
from tensorflow.keras.layers import Bidirectional, Dense, Input, Embedding, Conv1D, GlobalMaxPooling1D, Dropout
#from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras.layers import LSTM


from tensorflow.keras import regularizers
from tensorflow.keras.models import *

Attention = LuongAttention(20, 'dot')

def hybrid_model(kernel_size = 3, input_dim = vocab_size, output_dim=100, max_length = None, emb_matrix = None):

    inp = Input(shape=(max_length,))
    x = Embedding(input_dim=input_dim,
                  output_dim=output_dim,
                  input_length=max_length,
                  weights = [emb_matrix],
                  trainable = False)(inp)

    #x = Bidirectional(LSTM(100, stateful=False, return_sequences = True))(x)
    x = Bidirectional(LSTM(100, return_sequences=True, activation='tanh', recurrent_activation='sigmoid', use_bias=True))(x)
    x = LSTM(100, return_sequences=True, activation='tanh', recurrent_activation='sigmoid', use_bias=True)(x)
    # x = tf.keras.activations.sigmoid(x)
    #x = LSTM(100, stateful=False, return_sequences = True)(x)
    # x = tf.keras.activations.sigmoid(x)

    x = Conv1D(50, kernel_size, activation='relu')(x)

    print("conv shape", x.shape)
    x = Attention(x,x)[0]
    x = Dense(50, activation="relu", kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4))(x)
    # x = tf.keras.activations.relu(x)
    # regularizer = regularizers.L1L2(l1=1e-5, l2=1e-4)
    # regularizer(x)

    x = GlobalMaxPooling1D()(x)
    x = Dense(250, activation="relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(50, activation="relu")(x)
    x = Dropout(0.5)(x)
    print("before output shape", x.shape)
    outp = Dense(2, activation="sigmoid")(x)
    # outp = Dense(1, activation="sigmoid")(x)

    model = Model(inputs=inp, outputs=outp)
    model.compile( loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

    return model

model= hybrid_model(kernel_size = 3, input_dim = vocab_size, output_dim=embed_dim, max_length = max_length, emb_matrix = embed_matrix)

model.summary()

conv shape (None, 5610, 50)
before output shape (None, 50)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 5612)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 5612, 300)      │     13,168,500 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional             │ (None, 5612, 200)      │        320,800 │ embedding[0][0]        │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_1 (LSTM)             │ (None, 5612, 100)      │        120,400 │ bidirectional[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d (Conv1D)           │ (None, 5610, 50)       │         15,050 │ lstm_1[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ luong_attention           │ [(None, 5610, 50),     │              0 │ conv1d[0][0],          │
│ (LuongAttention)          │ (None, 5610, 5610)]    │                │ conv1d[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 5610, 50)       │          2,550 │ luong_attention[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_max_pooling1d      │ (None, 50)             │              0 │ dense[0][0]            │
│ (GlobalMaxPooling1D)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 250)            │         12,750 │ global_max_pooling1d[… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 250)            │              0 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 50)             │         12,550 │ dropout[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 50)             │              0 │ dense_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 2)              │            102 │ dropout_1[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 13,652,702 (52.08 MB)

 Trainable params: 484,202 (1.85 MB)

 Non-trainable params: 13,168,500 (50.23 MB)

In [38]:
epochs = 2 # 2 epochs in 85 mins on Macbook air 2023 24G RAM
# 35 mins with Colab T4
batch_size = 32
r = model.fit(X_train, y_train, epochs=epochs, batch_size=8, validation_data=(X_val, y_val))

Epoch 1/2
1599/1599 ━━━━━━━━━━━━━━━━━━━━ 1033s 644ms/step - accuracy: 0.8709 - loss: 0.3414 - val_accuracy: 0.9550 - val_loss: 0.1612
Epoch 2/2
1599/1599 ━━━━━━━━━━━━━━━━━━━━ 1083s 678ms/step - accuracy: 0.9554 - loss: 0.1423 - val_accuracy: 0.9628 - val_loss: 0.1002


# Prediction

In [39]:
#Prediction on the test data # 3mins
pred = model.predict(X_test)
print(pred)

125/125 ━━━━━━━━━━━━━━━━━━━━ 48s 375ms/step
[[0.9239314  0.11464568]
 [0.923564   0.11508633]
 [0.923438   0.11524092]
 ...
 [0.7314763  0.3037053 ]
 [0.92397743 0.11459427]
 [0.1532493  0.99994063]]


In [40]:
y_pred = np.zeros(len(pred))
for item in range(len(pred)):
  y_pred[item] = list(pred[item]).index(max(pred[item]))
  # y_pred[item] = 0 if pred[item]<0.5 else 1

print(y_pred)

[0. 0. 0. ... 0. 0. 1.]


In [41]:
import sklearn

report = sklearn.metrics.classification_report(y_test, y_pred)
print(report)
accuracy = sklearn.metrics.accuracy_score(y_test, y_pred)
precision = sklearn.metrics.precision_score(y_test, y_pred)
recall = sklearn.metrics.recall_score(y_test, y_pred)
f1score = sklearn.metrics.f1_score(y_test, y_pred)
print("-----------------------")
print(accuracy)
print(precision)
print(recall)
print(f1score)

## 15 epochs
# precision    recall  f1-score   support

#            0       0.99      0.92      0.96      2002
#            1       0.93      0.99      0.96      1995

#     accuracy                           0.96      3997
#    macro avg       0.96      0.96      0.96      3997
# weighted avg       0.96      0.96      0.96      3997

# -----------------------
# 0.9582186639979985
# 0.9279026217228464
# 0.993483709273183
# 0.9595739530380054


# 2 epochs with Colab T4
#               precision    recall  f1-score   support

#            0       0.98      0.95      0.97      2002
#            1       0.95      0.98      0.97      1995

#     accuracy                           0.97      3997
#    macro avg       0.97      0.97      0.97      3997
# weighted avg       0.97      0.97      0.97      3997

# -----------------------
# 0.9654741055791843
# 0.9531478770131772
# 0.9789473684210527
# 0.9658753709198813

              precision    recall  f1-score   support

           0       0.98      0.95      0.97      2002
           1       0.95      0.98      0.97      1995

    accuracy                           0.97      3997
   macro avg       0.97      0.97      0.97      3997
weighted avg       0.97      0.97      0.97      3997

-----------------------
0.9654741055791843
0.9531478770131772
0.9789473684210527
0.9658753709198813


In [ ]:
sdfsdf()

## Reddit+FB

In [ ]:
epochs = 15
batch_size = 16
r = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val))

In [ ]:
model.save('/trained_model_Reddit_Facebook.h5')

### Prediction

In [ ]:
#Prediction on the test data
pred = model.predict(X_test)
print(pred)

In [ ]:
y_pred = np.zeros(len(pred))
for item in range(len(pred)):
  y_pred[item] = list(pred[item]).index(max(pred[item]))
  # y_pred[item] = 0 if pred[item]<0.5 else 1

print(y_pred)

In [ ]:
import sklearn

report = sklearn.metrics.classification_report(y_test, y_pred)
print(report)
accuracy = sklearn.metrics.accuracy_score(y_test, y_pred)
precision = sklearn.metrics.precision_score(y_test, y_pred)
recall = sklearn.metrics.recall_score(y_test, y_pred)
f1score = sklearn.metrics.f1_score(y_test, y_pred)
print("-----------------------")
print(accuracy)
print(precision)
print(recall)
print(f1score)

#       precision    recall  f1-score   support

#            0       0.99      0.92      0.96      2002
#            1       0.93      0.99      0.96      1995

#     accuracy                           0.96      3997
#    macro avg       0.96      0.96      0.96      3997
# weighted avg       0.96      0.96      0.96      3997

# -----------------------
# 0.9582186639979985
# 0.9279026217228464
# 0.993483709273183
# 0.9595739530380054